In [1]:
from data.data import COVID19India

In [2]:
cin = COVID19India()

df = cin.moh_data(save=False)

In [5]:
df.tail()

,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
21,Ladakh,13,0,0,0
22,Uttar Pradesh,34,1,11,0
23,Uttarakhand,3,1,0,0
24,West Bengal,9,0,0,1
25,Total number of confirmed cases in India,519,43,41,9


df

In [60]:
df2 = cin.jhu_india_data()

In [61]:
df2.tail()

,date,confirmed,recovered,deaths
57,2020-03-19,194.0,15.0,4.0
58,2020-03-20,244.0,20.0,5.0
59,2020-03-21,330.0,23.0,4.0
60,2020-03-22,396.0,27.0,7.0
61,2020-03-23,396.0,27.0,7.0


In [4]:
import pandas as pd

In [5]:
from pathlib import Path

In [6]:
import pandas as pd
import numpy as np
import requests

In [7]:
link = "https://www.mohfw.gov.in/"

In [8]:
dfs = pd.read_html(link)

In [11]:
df = dfs[-1]

In [14]:
df[:-1]

,S. No.,Name of State / UT,Total Confirmed cases (Indian National),Total Confirmed cases ( Foreign National ),Cured/Discharged/Migrated,Death
0,1,Andhra Pradesh,11,0,1,0
1,2,Bihar,3,0,0,1
2,3,Chhattisgarh,3,0,0,0
3,4,Delhi,34,1,6,1
4,5,Goa,3,0,0,0
5,6,Gujarat,37,1,0,2
6,7,Haryana,16,14,11,0
7,8,Himachal Pradesh,3,0,0,1
8,9,Karnataka,41,0,3,1
9,10,Kerala,110,8,4,0


markdown_file = open('help.md')
file_content = markdown_file.read()
md = markdown.Markdown(extensions=['tables', 'codehilite', 'fenced_code'])
pg = md.convert(file_content)

In [4]:
url1 = "https://api.covid19india.org/state_district_wise.json"
url2 = "https://www.covid19india.org/"

In [10]:
import json

In [13]:
data = json.loads(requests.get(url1).content.decode('utf-8'))

In [44]:
Values = []
for k in key1:
#     v = []
#     v.append(k)
    key2 = data[k]['districtData'].keys()
    for k2 in key2:
        c = list(data[k]['districtData'][k2].values())
        v = [k, k2, c[1], c[0], c[2], c[4] ]
        Values.append(v)
data = pd.DataFrame(Values, columns=['State_UT', 'District', 'Confirmed', 'Active', 'Deaths', 'Recovered'])

In [45]:
data

,State_UT,District,Confirmed,Active,Deaths,Recovered
0,Kerala,Thrissur,3,1,0,2
1,Kerala,Alappuzha,1,0,0,1
2,Kerala,Kasaragod,39,38,0,1
3,Kerala,Pathanamthitta,12,12,0,0
4,Kerala,Kannur,14,13,0,1
...,...,...,...,...,...,...
123,Bihar,Patna,3,3,0,0
124,Manipur,Imphal West,1,1,0,0
125,Mizoram,Aizawl,1,1,0,0
126,Goa,Goa,3,3,0,0


In [49]:
from bs4 import BeautifulSoup

In [50]:
content = requests.get(url2).content.decode('utf-8')

In [52]:
soup = BeautifulSoup(content, 'html.parser')

In [54]:
data_india = "https://www.covid19india.org/india.json"
data_data = "https://api.covid19india.org/data.json"
data_website = "https://api.covid19india.org/website_data.json"

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><link href="/favicon.ico" rel="icon"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="#000000" name="theme-color"/><meta content="Tracking the coronavirus cases in India during the outbreak of the disease in 2020." name="COVID-19 Tracker | India"/><link href="/logo192.png" rel="apple-touch-icon"/><link href="/manifest.json" rel="manifest"/><link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/><link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/><link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/><link href="/site.webmanifest" rel="manifest"/><title>COVID-19 Tracker | India</title><meta content="COVID-19 Tracker | India" name="title"/><meta content="Tracking the coronavirus cases in India during the outbreak of the disease in 2020." name="description"/><meta content="website" property="og:type"/><meta content="https://www.covid

In [53]:
pd.read_html(soup)

TypeError: 'NoneType' object is not callable